In [2]:
## inline figures and output
%matplotlib inline
## clear the workspace
%reset -f

In [2]:
## install packages
import sys
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install h5py

In [3]:
##################################################################
## MODULES
##################################################################
import os
import argparse
import numpy as np
import cmasher as cmr # https://cmasher.readthedocs.io/user/diverging.html

from matplotlib.gridspec import GridSpec

## load old user defined modules
from the_useful_library import *
from the_plotting_library import *
from the_dynamo_library import *
from the_matplotlib_styler import *

In [4]:
filepath_base   = "/Users/dukekriel/Documents/Projects/Turbulent-Dynamo/data"
filepath_data   = createFilePath([filepath_base, "fixed_Rm"])
filepath_frames = createFilePath([filepath_base, "vis_folder"])
createFolder(filepath_frames)

	/Users/dukekriel/Documents/Projects/Turbulent-Dynamo/data/vis_folder



# Functions

In [5]:
def plotSpectra(
        ax, color, time_index,
        k, power,
        fit_k, fit_power, scale
    ):
    ## plot obs spectra
    ax.plot(k[time_index], power[time_index], color=color, ls="", marker=".", markersize=8)
    ## plot fitted spectra
    ax.plot(fit_k[time_index], fit_power[time_index], color=color, linestyle="--", dashes=(5, 2.5), linewidth=2)
    ## plot measured scale
    ax.axvline(x=scale[time_index], color=color, ls="-.")

def adjustFig(
        ax, label,
        x_min=0.5, x_max=250,
        y_min=1e-12, y_max=10
    ):
    ## label axes
    ax.set_xlabel(r"$k$")
    ax.set_ylabel(r"$\mathcal{P}$")
    ## log-log axis
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ## add fig-label
    ax.text(
        0.05, 0.95, label,
        va="top", ha="left", transform=ax.transAxes, fontsize=16,
        bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round', alpha=0.85)
    )

def addLegend(ax, artists, legend_labels, colors=["k"], bool_place_top=False, title=None):
    ## check that the inputs are correct length
    if len(artists) < len(legend_labels): artists.extend( artists[0] * len(legend_labels) )
    if len(colors) < len(legend_labels): colors.extend( colors[0] * len(legend_labels) )
    ## useful lists
    list_markers   = ["o", "s", "D", "^", "v"] # list of marker styles
    list_lines     = ["-", "--", "-."] # list of line styles
    legend_artists = [] # iniialise list of artists for legend
    ## create legend artists
    for artist, color in  zip(artists, colors):
        if artist in list_markers: legend_artists.append( Line2D([0], [0], marker=artist,    color=color, linewidth=0) )
        elif artist in list_lines: legend_artists.append( Line2D([0], [0], linestyle=artist, color=color, linewidth=2) )
        else: raise Exception("Artist '{}' is not a valid marker- or line-style.")
    ## create legend
    if bool_place_top: legend = plt.legend(
            legend_artists, legend_labels, title=title,
            frameon=True, loc="lower center", bbox_to_anchor=(0.5, 1.0),
            columnspacing=1.0, handletextpad=0.4,
            facecolor="white", framealpha=0.5, fontsize=14, ncol=len(legend_artists)
        )
    else: legend = plt.legend(
            legend_artists, legend_labels, title=title,
            frameon=True, loc="center left", bbox_to_anchor=(1.0, 0.5),
            labelspacing=1.2,
            facecolor="white", framealpha=0.5, fontsize=14, ncol=1
        )
    ## add legend
    ax.add_artist(legend)

# Plot spectra

In [8]:
## initialise side-by-side figure
fig = plt.figure(figsize=(10,4), constrained_layout=True, dpi=200)
fig_grids = GridSpec(ncols=2, nrows=1, figure=fig)
ax0 = fig.add_subplot(fig_grids[0])
ax1 = fig.add_subplot(fig_grids[1])
## plot spectra
sim_folder = "Pm125"
list_res = [72, 144, 288, 576]
time_index = 50
## extract colours from Cmasher's colormap
num_cols = len(list_res)
cmasher_colormap = plt.get_cmap("cmr.tropical", num_cols)
my_colormap = cmasher_colormap(np.linspace(0, 1, num_cols))
print("Plotting spectra...")
for res, plot_index in zip(list_res, range(len(list_res))):
    ## ####################
    ## LOAD SPECTRA OBJECT
    ## ####################
    ## if the file exists, then read it in
    filename = createFilePath([filepath_data, str(res), sim_folder, "spectra_obj.pkl"])
    print("\t> Loading: " + filename)
    if os.path.isfile(filename):
        with open(filename, "rb") as input:
            spectra_obj = pickle.load(input)
    else: raise Exception("\t\t> No spectra object found.")
    ## ####################
    ## PLOT SPECTRA
    ## ####################
    plotSpectra(
        ax         = ax0,
        color      = my_colormap[plot_index],
        time_index = time_index,
        k          = spectra_obj.vel_k,
        power      = spectra_obj.vel_power,
        fit_k      = spectra_obj.vel_fit_k,
        fit_power  = spectra_obj.vel_fit_power,
        scale      = spectra_obj.k_nu
    )
    plotSpectra(
        ax         = ax1,
        color      = my_colormap[plot_index],
        time_index = time_index,
        k          = spectra_obj.mag_k,
        power      = spectra_obj.mag_power,
        fit_k      = spectra_obj.mag_fit_k,
        fit_power  = spectra_obj.mag_fit_power,
        scale      = spectra_obj.k_eta
    )
print("Labeling figure...")
adjustFig(ax0, "vel-spectra", y_min=10**(-12) ,y_max=10)
adjustFig(ax1, "mag-spectra", y_min=10**(-13) ,y_max=10**(-4))
addLegend(
    ax             = ax1,
    title          = sim_folder,
    artists        = ["o", "--", "-."],
    colors         = ["k"],
    legend_labels  = ["spectra data", "spectra fits", r"$k_\nu$ or $k_\eta$"],
    bool_place_top = False
)
addLegend(
    ax             = ax1,
    artists        = ["-"],
    colors         = my_colormap[:num_cols],
    legend_labels  = [ r"${}^3$".format(str(res)) for res in list_res ],
    bool_place_top = True
)
## save plot
print("Saving figure...")
name_fig = createFilePath([filepath_frames, sim_folder+"_spectra_res.pdf"])
plt.savefig(name_fig)
print("\t> Figure saved: " + name_fig)
## close plot
plt.close(fig)

Plotting spectra...
	> Loading: /Users/dukekriel/Documents/Projects/Turbulent-Dynamo/data/fixed_Rm/72/Pm125/spectra_obj.pkl
	> Loading: /Users/dukekriel/Documents/Projects/Turbulent-Dynamo/data/fixed_Rm/144/Pm125/spectra_obj.pkl
	> Loading: /Users/dukekriel/Documents/Projects/Turbulent-Dynamo/data/fixed_Rm/288/Pm125/spectra_obj.pkl
	> Loading: /Users/dukekriel/Documents/Projects/Turbulent-Dynamo/data/fixed_Rm/576/Pm125/spectra_obj.pkl
Labeling figure...
Saving figure...
	> Figure saved: /Users/dukekriel/Documents/Projects/Turbulent-Dynamo/data/vis_folder/Pm125_spectra_res.pdf
